In [1]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import re
import uuid 
import json
import sqlite3

pd.set_option('display.max_rows', 900)

In [2]:
class BETS():
    def __init__(self):
        load_dotenv()
        self.clientId = os.getenv('CLIENT_ID')
        self.clientPw = os.getenv('CLIENT_PW')
        self.baseUrl = "https://api.tournamentsoftware.com/1.0"
    
    def make_call(self,url):
        response = requests.get(url, auth=HTTPBasicAuth(self.clientId, self.clientPw))
        return response

    #   USE SEARCH QUERY '?q=U11 Gold'
    #   Filter by dates
    #   Create lists of cool tourney codes.
    def search_tournaments(self,dateStr):
        splitDate = dateStr.split('/')
        refDate = splitDate[2] + '-' + splitDate[1] + '-' + splitDate[0]
        url = self.baseUrl+f"/Tournament?list=1&refdate={refDate}&pagesize=1000" 
        return self.make_call(url)
    
    def get_tournament_details(self,tCode):
        url = self.baseUrl+f"/Tournament/{tCode}"
        return self.make_call(url)

    def get_matches(self,tCode,date):
        dateQuery = date.strftime("%Y%m%d")
        url = self.baseUrl+f"/Tournament/{tCode}/Match/{dateQuery}"
        return self.make_call(url)
    
    def wanted_rounds(self,roundCaredAbout):
        allRounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'Final']
        if roundCaredAbout == 'Winner':
            return ['Final']
        elif roundCaredAbout in allRounds:
            index = allRounds.index(roundCaredAbout)
            return allRounds[index:]
        else:
            return ['','','','','','','']


In [3]:
def processXMLList(list):
    out = []
    for xml in list:
        out.append(xml.text)
    return out

def get_exclude_list():
    tabuTitles = ['invite','(1000ie)','masters','para','yonex','league','regional','schools','graded','rising','handicap']
    return tabuTitles

def get_tourney_deets(title):
    title = title.lower()
    age = ''
    level = ''

    if 'tier 4' in title:
        level = 'Tier 4'
    if 'bronze' in title:
        level = 'Bronze'
    if 'silver' in title:
        level = 'Silver'
    if 'gold' in title:
        level = 'Gold'
    if 'national' in title:
        level = 'Nationals'
    if 'restricted' in title:
        level = 'Restricted'
    
    ageGroupRegex = re.compile('u(1[0-9])')
    ageMatch = re.search(ageGroupRegex,title)
    if ageMatch:
        age = ageMatch.group().upper()
    elif 'senior' in title:
        age = 'Senior'

    if level and not age: #Assume senior if not in title
        age = 'Senior'

    return age,level

def checkMatch(matchInfo):
    for team in ['Team1','Team2']:
        id = matchInfo[f'{team}_MemberID'] 
        fn = matchInfo[f'{team}_Firstname']
        sn = matchInfo[f'{team}_Lastname'] 
        g = matchInfo[f'{team}_GenderID'] 

        if not (id and fn and sn and g):
            #print(f'Error in info for match: \n {matchInfo}')
            return False
        
    return True

#Both inputs as strings, searchStart = d/m/y, searchText = 'U11 Gold'
def get_tournament_results(bets,searchStartStr):
    tabuList = get_exclude_list()
    searchStart = datetime.strptime(searchStartStr,"%d/%m/%Y")
    tourneysDf = pd.DataFrame(columns=['Name','Code','StartDate','EndDate'])
    tourneysToAdd = []

    out = bets.search_tournaments(searchStartStr).content
    soup = BeautifulSoup(out,'xml')
    tournaments = soup.find_all('Tournament')
    for i in range(0,len(tournaments)):  
        name = tournaments[i].find('Name').text
        code = tournaments[i].find('Code').text
        startDate = datetime.fromisoformat(tournaments[i].find('StartDate').text)
        endDate = datetime.fromisoformat(tournaments[i].find('EndDate').text)
        tType = tournaments[i].find('TypeID').text
        
        goodTourney = True
        if not tType == '0':
            goodTourney = False
        
        for t in tabuList:
            if t in name.lower():
                goodTourney = False
        if not any(existing_tourney['Name'] == name for existing_tourney in tourneysToAdd) and goodTourney:
            tourney = {
                "Name": name,
                "Code": code,
                "StartDate": startDate,
                "EndDate": endDate,
            }

            tourneysToAdd.append(tourney)

    tourneysDf = pd.concat([tourneysDf, pd.DataFrame(tourneysToAdd)], ignore_index=True)
    searchUntil = datetime.today() - timedelta(days = 2)
    searchFrom = searchStart

    
    # Apply condition and filter rows
    tourneysInRangeDf = tourneysDf[(tourneysDf['StartDate'] <= searchUntil)].copy()

    linksDf = tourneysInRangeDf[['Name','Code','StartDate']]

    def code_to_link(code):
        return f'https://be.tournamentsoftware.com/tournament/{code}'
    
    linksDf['Code'] = linksDf['Code'].apply(code_to_link)


    #Itterate through Tournaments
    match_data = []
    for index, row in tourneysInRangeDf.iterrows():
        dtCurrentDay = row['StartDate']
        dtEndDate = row['EndDate']
        code = row['Code']
        tName = row['Name']
        

        
        #Itterate through Days With Matches.
        allMatchesXML = [] 
        while dtCurrentDay <= dtEndDate:
            tDetails = bets.get_matches(row['Code'],dtCurrentDay)
            matchesXML = tDetails.content
            allMatchesXML.append(matchesXML)
            dtCurrentDay = dtCurrentDay + timedelta(days=1)

        strCurrentDay = datetime.strftime(dtCurrentDay,'%d/%m/%Y')
        tAge,tLevel = get_tourney_deets(tName)
        # Iterate over each Match element
        if not (tAge and tLevel): # dont append if not interested
            print(f'Did not append {tName}.')
        else:
            for matchesXML in allMatchesXML:
                soup = BeautifulSoup(matchesXML,'xml')
                for match in soup.find_all('Match'):

                    #check if match went ahead:
                    if not match.find('ScoreStatus').text == '0':
                        pass
                    else:
                        try:
                            roundName = match.find('RoundName').text
                        except:
                            roundName = 'Not Found'
                            print('no round name for match:',match)
                        match_info = {
                            'Tournament': tName,
                            'Date': strCurrentDay,
                            'Age': tAge,
                            'Level':tLevel,
                            'Code': match.find('Code').text,
                            'Winner': match.find('Winner').text,
                            'ScoreStatus': match.find('ScoreStatus').text,
                            'RoundName': roundName,
                            'EventName': match.find('EventName').text,
                        }
                        # Extract team and player details
                        teams = []
                        for team in ['Team1', 'Team2']:
                            teamXML = match.find(team)
                            team_info = {}
                            #find player id's and if singles
                            team_info[f'{team}_MemberID'] = processXMLList(teamXML.find_all('MemberID'))
                            team_info[f'{team}_Firstname'] = processXMLList(teamXML.find_all('Firstname'))
                            team_info[f'{team}_Lastname'] = processXMLList(teamXML.find_all('Lastname'))
                            team_info[f'{team}_GenderID'] = processXMLList(teamXML.find_all('GenderID'))

                            teams.append(team_info)

                        match_info.update(teams[0])
                        match_info.update(teams[1])

                        # Extract sets scores
                        sets = match.find('Sets')
                        set_scores = []
                        if sets:
                            for set_ in sets.find_all('Set'):
                                set_scores.append({
                                    'Set_Team1': set_['Team1'],
                                    'Set_Team2': set_['Team2']
                                })

                            match_info['Set_Scores'] = set_scores


                        # Append the structured match information to the list
                        if checkMatch(match_info): # Check everything is correct
                            match_data.append(match_info)

                # Convert the list to a DataFrame
                
    df = pd.DataFrame(match_data)
    return df,linksDf


In [4]:
bets = BETS()
matchData,linksDf = get_tournament_results(bets,'01/01/2024') #d m y

/tmp/ipykernel_14825/2530594073.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linksDf['Code'] = linksDf['Code'].apply(code_to_link)


Did not append Essex Futures 2* - Colchester 2025.
Did not append South West Disability Tournament 2025.
Did not append Sheffield Disability Tournament 2025.
Did not append Herts B Tournament - 1.
Did not append U17-U19 EJPP Camp January 2025.
Did not append Kent Junior Futures Jan 2025.
Did not append U13 EJPP Camp January 2025.
Did not append U15 EJPP Camp January 2025.
Did not append Sussex 3 Series + (Crawley).
Did not append Essex Futures 2* - Chelmsford Nov 2024.
Did not append Southport C Tournament 2024.
Did not append Guernsey Closed Tournament 2024.
Did not append PETRONAS Malaysia Open 2025 1000IE.
Did not append U13 Babolat Cup 2024.
Did not append Essex Futures 1* - East Thurrock Nov 2024.
Did not append Essex Futures 2* - Redbridge Oct 2024.
no round name for match: <Match><Code>282</Code><Winner>1</Winner><ScoreStatus>0</ScoreStatus><MatchTime>2024-10-20T09:30:00</MatchTime><EventCode>4</EventCode><EventName>Open Doubles</EventName><DrawCode>16</DrawCode><DrawName>Open D

In [5]:
matchData.head(1)

,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,Team1_Firstname,Team1_Lastname,Team1_GenderID,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores
0,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,175,2,0,R1,Open Singles,[1358872],[Punit],[Kishorekumar],[1],[1396470],[Rhys Andrew],[Williams],[1],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."


In [6]:
# Format names
for team in ['Team1','Team2']:
    namesFormatted = []
    for index, row in matchData.iterrows():
        firstNames = row[f'{team}_Firstname']
        secondNames = row[f'{team}_Lastname']

        if not (len(firstNames) == len(secondNames)):
            raise ValueError(f'There are not matching numbers of first and second names when formatting.\n See below for row \n {row}')

        playerNames = []
        if len (firstNames) > 1: #Doubles
            
            for i in range(0,len(firstNames)):
                playerNames.append(firstNames[i] + ' ' +  secondNames[i])

        else: #Singles
            if firstNames and secondNames:
                playerNames.append(firstNames[0] + ' ' +  secondNames[0])
            else:
                raise ValueError(f'Error parsing names from row: \n {row}')

        namesFormatted.append(playerNames)

    matchData[f'{team}_Names'] = namesFormatted


## Drop the original columns
#df = df.drop(['Col1', 'Col2'], axis=1)
#
#print(df)



In [7]:
#Format Event
matchData['Event'] = ''
matchData['Star'] = ''

for index,row in matchData.iterrows():
    strEvent = row['EventName'].lower().strip().replace('/','')
 
    allEvents = ['os','od',
                 'ms','md','bs','bd',
                 'ws','wd','gs','gd',
                 'xd'] 

    eventOut = ''

    isStar = False
    if '*' in strEvent: # Check if its a star event
        isStar = True

    for event in allEvents:
        if (event.lower() in strEvent.lower()): # handle two letter abreviation
            eventOut = event.upper()
    


    else: #Handle inputs such as 'open singles'
        if 'single' in strEvent:
            if ('men' in strEvent) or ('open' in strEvent) or ('boy' in strEvent):
                eventOut = 'OS'
            else:
                eventOut = 'WS'

        elif 'mix' in strEvent:
            eventOut = 'XD'

        elif 'double' in strEvent:
            if ('men' in strEvent) or ('open' in strEvent) or ('boy' in strEvent):
                eventOut = 'OD'
            else:
                eventOut = 'WD'
        

    if eventOut: # Append the abreviated event
        row['Event'] = eventOut 

        if isStar: #Add star flag if it was
            row['Star'] = True
        else:
            row['Star'] = False

    else:
        print('No known event for event:',row['EventName'])



#Format Gender
def replace_gender(value):
    if isinstance(value, list):  # Check if the entry is a list
        return ['M' if v == '1' else 'F' for v in value]
    return 'M' if value == '1' else 'F'

def replace_round(value):
    numRegex = re.compile('\d+')
    if 'R' in value:
        roundNum = re.findall(numRegex,value)[0]
        return 'Round ' + roundNum
    if value == 'SF':
        return 'Semi-Final'
    if value == 'QF':
        return 'Quarter-Final'
    if value == 'Final':
        return value

# Apply the replacement
for team in ['Team1','Team2']:
    matchData[f'{team}_Gender'] = matchData[f'{team}_GenderID'].apply(replace_gender)

# Create list of all ID's for quick indexing
allIds = []
for index,row in matchData.iterrows():
    rowIds = []
    for team in ['Team1','Team2']:
        for id in row[f'{team}_MemberID']:
            rowIds.append(id)
    allIds.append(rowIds)
matchData['PID'] = allIds

matchData['Round'] = matchData['RoundName'].apply(replace_round)

matchData#.head(5)


,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,...,Team2_GenderID,Set_Scores,Team1_Names,Team2_Names,Event,Star,Team1_Gender,Team2_Gender,PID,Round
0,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,175,2,0,R1,Open Singles,[1358872],...,[1],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set...",[Punit Kishorekumar],[Rhys Andrew Williams],OS,False,[M],[M],"[1358872, 1396470]",Round 1
1,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,184,1,0,R1,Open Singles,[1298311],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set...",[Shaurya Singh],[Ayush Iyengar],OS,False,[M],[M],"[1298311, 1350351]",Round 1
2,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,193,1,0,R1,Open Singles,[1346258],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set...",[Kevin Tao],[Anish Reddy Kokatam],OS,False,[M],[M],"[1346258, 1379398]",Round 1
3,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,139,1,0,R1,Open Singles,[1370416],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set...",[Suchir Krishna Addagondla],[Fausto Ku],OS,False,[M],[M],"[1370416, 1343190]",Round 1
4,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,202,1,0,R1,Open Singles,[1360210],...,[1],"[{'Set_Team1': '21', 'Set_Team2': '17'}, {'Set...",[Suhas Govindarajula],[Ethan Feakin],OS,False,[M],[M],"[1360210, 1340995]",Round 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35512,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,361,1,0,R3,XD U15,"[1343551, 1366097]",...,"[1, 2]","[{'Set_Team1': '23', 'Set_Team2': '21'}]","[Akhil Luke Gibi, Casey Luk]","[Tuvyojas Didwania, Siddhi Lodha]",XD,False,"[M, F]","[M, F]","[1343551, 1366097, 1339298, 1360449]",Round 3
35513,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,365,2,0,R3,XD U15,"[1341330, 1375119]",...,"[1, 2]","[{'Set_Team1': '11', 'Set_Team2': '21'}]","[Aarav Kotian, Francesca Lim]","[Advait Neelakandan, Sarah Devapragasam]",XD,False,"[M, F]","[M, F]","[1341330, 1375119, 1342449, 1296869]",Round 3
35514,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,345,1,0,R3,XD U15,"[1338216, 1316620]",...,"[1, 2]","[{'Set_Team1': '21', 'Set_Team2': '6'}]","[James Ng, Livia Sebestova]","[Roneth Abhayaratne, Tanu Malakannavar]",XD,False,"[M, F]","[M, F]","[1338216, 1316620, 1380502, 1311130]",Round 3
35515,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,349,1,0,R3,XD U15,"[1381811, 1375118]",...,"[1, 2]","[{'Set_Team1': '21', 'Set_Team2': '16'}]","[Jasper Chan, Reika Lim]","[Akash Raviprakash, Aashritha Kolluru]",XD,False,"[M, F]","[M, F]","[1381811, 1375118, 1379608, 1365478]",Round 3


In [8]:
explosionTest = matchData.explode('PID')

In [9]:
#For later: 'Tournament','Date','Age','Level', 'RoundName'
wantedCols = ['PID','Tournament','Date','Age','Level','Event','Round','Winner','Team1_MemberID','Team1_Names','Team1_Gender','Team2_MemberID','Team2_Names','Team2_Gender','Set_Scores']

explosionTest = explosionTest[wantedCols]
explosionTest

,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,1358872,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,[1358872],[Punit Kishorekumar],[M],[1396470],[Rhys Andrew Williams],[M],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."
0,1396470,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,[1358872],[Punit Kishorekumar],[M],[1396470],[Rhys Andrew Williams],[M],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."
1,1298311,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1298311],[Shaurya Singh],[M],[1350351],[Ayush Iyengar],[M],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set..."
1,1350351,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1298311],[Shaurya Singh],[M],[1350351],[Ayush Iyengar],[M],"[{'Set_Team1': '21', 'Set_Team2': '14'}, {'Set..."
2,1346258,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,[1346258],[Kevin Tao],[M],[1379398],[Anish Reddy Kokatam],[M],"[{'Set_Team1': '21', 'Set_Team2': '12'}, {'Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35515,1365478,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Round 3,1,"[1381811, 1375118]","[Jasper Chan, Reika Lim]","[M, F]","[1379608, 1365478]","[Akash Raviprakash, Aashritha Kolluru]","[M, F]","[{'Set_Team1': '21', 'Set_Team2': '16'}]"
35516,1338216,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,1,"[1338216, 1316620]","[James Ng, Livia Sebestova]","[M, F]","[1343551, 1366097]","[Akhil Luke Gibi, Casey Luk]","[M, F]","[{'Set_Team1': '21', 'Set_Team2': '11'}]"
35516,1316620,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,1,"[1338216, 1316620]","[James Ng, Livia Sebestova]","[M, F]","[1343551, 1366097]","[Akhil Luke Gibi, Casey Luk]","[M, F]","[{'Set_Team1': '21', 'Set_Team2': '11'}]"
35516,1343551,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,1,"[1338216, 1316620]","[James Ng, Livia Sebestova]","[M, F]","[1343551, 1366097]","[Akhil Luke Gibi, Casey Luk]","[M, F]","[{'Set_Team1': '21', 'Set_Team2': '11'}]"


In [10]:
def make_player_in_team1(row):
    if str(row['PID']) in row['Team2_MemberID']:  # If the player is in Team2, swap
        # Swap all relevant columns
        row['Team1_MemberID'], row['Team2_MemberID'] = row['Team2_MemberID'], row['Team1_MemberID']
        row['Team1_Names'], row['Team2_Names'] = row['Team2_Names'], row['Team1_Names']
        row['Team1_Gender'], row['Team2_Gender'] = row['Team2_Gender'], row['Team1_Gender']
        
        # Swap set scores (Team1 ↔ Team2)
        if isinstance(row['Set_Scores'], list):
            swapped_scores = []
            for score in row['Set_Scores']:
                # Ensure 'Set_Team1' and 'Set_Team2' keys exist before swapping
                swapped_scores.append({'Set_Team1': score['Set_Team2'], 'Set_Team2': score['Set_Team1']})
            row['Set_Scores'] = swapped_scores
        
        # Update winner: If originally Team2 won (Winner = 2), set Winner = 1, and vice versa
        prevWinner = int(row['Winner'])
        row['Winner'] = 1 if prevWinner == 2 else 2

    return row

# Apply the function to each row
preCorrectedDf = explosionTest.copy()
correctedDf = explosionTest.apply(make_player_in_team1, axis=1)


In [11]:
correctedDf.head(1)

,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,1358872,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,[1358872],[Punit Kishorekumar],[M],[1396470],[Rhys Andrew Williams],[M],"[{'Set_Team1': '17', 'Set_Team2': '21'}, {'Set..."


### The following is for database dramas

In [13]:
# Copy DataFrame to avoid modifying the original
df = explosionTest.copy()
db_name = 'be-match-archive.db'
# Function to generate unique MatchID
def generate_match_id(row):
    date = row['Date'].replace('/','')
    
    # Get first player ID from Team1_MemberID (assuming it's a list)
    first_team1_id = str(row['PID'])
    
    # Convert Team2_MemberID to a string (flatten if it's a list)
    team2_id = ''.join(map(str, row['Team2_MemberID'])) if isinstance(row['Team2_MemberID'], list) else str(row['Team2_MemberID'])
    
    event = row['Event']
    
    # Concatenate all parts into a unique string
    return f"{date}_{first_team1_id}_{team2_id}_{event}"

# Apply function to create MatchID column
df.insert(0, 'MatchID', df.apply(generate_match_id, axis=1))

for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (list, dict))).any():
        df[col] = df[col].apply(json.dumps) 

def save_to_database(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Fetch existing MatchIDs from the database
    query = f"SELECT MatchID FROM {table_name}"
    try:
        existing_match_ids = pd.read_sql(query, conn)["MatchID"].tolist()
    except Exception as e:
        print(f"Table '{table_name}' does not exist or no MatchID column found. Proceeding with full insert.")
        existing_match_ids = []
 
    # Filter out matches that already exist
    df_filtered = df[~df["MatchID"].isin(existing_match_ids)]

    if not df_filtered.empty:
        df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Added {len(df_filtered)} new matches to '{table_name}' in '{db_name}'.")
    else:
        print("No new matches to add.")

    conn.close()

# Save DataFrame to the database
save_to_database(df, db_name, table_name="all_matches")





Table 'all_matches' does not exist or no MatchID column found. Proceeding with full insert.
Added 100578 new matches to 'all_matches' in 'be-match-archive.db'.


In [14]:
#Cull matches from db which are older than days_threshold days old.
def remove_old_matches(db_name, table_name, days_threshold=850):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Compute the threshold date
    threshold_date = (datetime.now() - timedelta(days=days_threshold)).strftime("%Y-%m-%d")  # Convert to YYYY-MM-DD

    # Convert Date column to proper format and delete old matches
    delete_query = f"""
    DELETE FROM {table_name} 
    WHERE DATE(substr(Date, 7, 4) || '-' || substr(Date, 4, 2) || '-' || substr(Date, 1, 2)) < ?
    """
    cursor.execute(delete_query, (threshold_date,))

    # Commit changes and close connection
    conn.commit()
    deleted_rows = cursor.rowcount
    conn.close()

    threshold_date = datetime.strptime(threshold_date,"%Y-%m-%d")
    threshold_date = datetime.strftime(threshold_date,'%d/%m/%Y')
    print(f"Removed {deleted_rows} old matches (before {threshold_date}) from '{table_name}'.")

# Example Usage:
# Remove matches older than 30 days
remove_old_matches(db_name, table_name="all_matches", days_threshold=750)


Removed 280 old matches (before 14/01/2023) from 'all_matches'.


In [15]:
# Load the DataFrame from SQLite
conn = sqlite3.connect(db_name)
df_loaded = pd.read_sql(f"SELECT * FROM all_matches", conn)

## Convert JSON strings back to lists/dicts
#for col in df_loaded.columns:
#    if df_loaded[col].apply(lambda x: isinstance(x, str) and x.startswith("[") or x.startswith("{") ).any():
#        df_loaded[col] = df_loaded[col].apply(json.loads)


conn.close()

df_loaded

,MatchID,PID,Tournament,Date,Age,Level,Event,Round,Winner,Team1_MemberID,Team1_Names,Team1_Gender,Team2_MemberID,Team2_Names,Team2_Gender,Set_Scores
0,27012025_1358872_1396470_OS,1358872,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,"[""1358872""]","[""Punit Kishorekumar""]","[""M""]","[""1396470""]","[""Rhys Andrew Williams""]","[""M""]","[{""Set_Team1"": ""17"", ""Set_Team2"": ""21""}, {""Set..."
1,27012025_1396470_1358872_OS,1396470,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1396470""]","[""Rhys Andrew Williams""]","[""M""]","[""1358872""]","[""Punit Kishorekumar""]","[""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""17""}, {""Set..."
2,27012025_1298311_1350351_OS,1298311,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1298311""]","[""Shaurya Singh""]","[""M""]","[""1350351""]","[""Ayush Iyengar""]","[""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""14""}, {""Set..."
3,27012025_1350351_1298311_OS,1350351,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,2,"[""1350351""]","[""Ayush Iyengar""]","[""M""]","[""1298311""]","[""Shaurya Singh""]","[""M""]","[{""Set_Team1"": ""14"", ""Set_Team2"": ""21""}, {""Set..."
4,27012025_1346258_1379398_OS,1346258,U17 Gold Milton Keynes January 2025,27/01/2025,U17,Gold,OS,Round 1,1,"[""1346258""]","[""Kevin Tao""]","[""M""]","[""1379398""]","[""Anish Reddy Kokatam""]","[""M""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""12""}, {""Set..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100293,30122023_1365478_13818111375118_XD,1365478,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Round 3,2,"[""1379608"", ""1365478""]","[""Akash Raviprakash"", ""Aashritha Kolluru""]","[""M"", ""F""]","[""1381811"", ""1375118""]","[""Jasper Chan"", ""Reika Lim""]","[""M"", ""F""]","[{""Set_Team1"": ""16"", ""Set_Team2"": ""21""}]"
100294,30122023_1338216_13435511366097_XD,1338216,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,1,"[""1338216"", ""1316620""]","[""James Ng"", ""Livia Sebestova""]","[""M"", ""F""]","[""1343551"", ""1366097""]","[""Akhil Luke Gibi"", ""Casey Luk""]","[""M"", ""F""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""11""}]"
100295,30122023_1316620_13435511366097_XD,1316620,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,1,"[""1338216"", ""1316620""]","[""James Ng"", ""Livia Sebestova""]","[""M"", ""F""]","[""1343551"", ""1366097""]","[""Akhil Luke Gibi"", ""Casey Luk""]","[""M"", ""F""]","[{""Set_Team1"": ""21"", ""Set_Team2"": ""11""}]"
100296,30122023_1343551_13382161316620_XD,1343551,Battledore Junior U15 Bronze,30/12/2023,U15,Bronze,XD,Final,2,"[""1343551"", ""1366097""]","[""Akhil Luke Gibi"", ""Casey Luk""]","[""M"", ""F""]","[""1338216"", ""1316620""]","[""James Ng"", ""Livia Sebestova""]","[""M"", ""F""]","[{""Set_Team1"": ""11"", ""Set_Team2"": ""21""}]"



**A list of available information for each match:**
- `MatchID` : Unique identifier for each match - in the form: Date_PID_OppID(s)_Event.
- `PID` : The players BE ID
- `Tournament` : Name of the tournament.
- `Date` : Match date (format: DD/MM/YYYY).
- `Age` : Age of tournament.
- `Level` : The level of the tournament (bronze, silver, ect...).
- `Event` : Dicipline: OS, WS, OD, WD or XD.
- `Round` : Round of the tournament played.
- `Winner` : The winning team. Matches formatted so the PID of the match row is always in team 1.
- `TeamX_MemberID` : IDs of Team X members.
- `TeamX_Names` : Names of Team X players.
- `TeamX_Gender` : Gender of Team X players
- `Set_Scores` : Scores for each set played. stored as a dict. see below for more.


In [16]:
roundCols = explosionTest['Round']
roundCols.unique()

array(['Round 1', 'Round 2', 'Round 3', 'Quarter-Final', 'Semi-Final',
       'Final', 'Round 32', 'Round 16', 'Round 4', 'Round 5', 'Round 64',
       None, 'Round 128', 'Round 6', 'Round 7'], dtype=object)

## Below is an example row

| MatchID                          | PID     | Tournament                               | Date       | Age  | Level | Event | Round   | Winner | Team1_MemberID | Team1_Names                   | Team1_Gender | Team2_MemberID | Team2_Names   | Team2_Gender | Set_Scores                                          |
|----------------------------------|---------|------------------------------------------|------------|------|-------|-------|---------|--------|----------------|--------------------------------|--------------|----------------|--------------|--------------|----------------------------------------------------|
| 27012025_1370416_1343190_OS     | 1370416 | U17 Gold Milton Keynes January 2025     | 27/01/2025 | U17  | Gold  | OS    | Round 1 | 1      | ["1370416"]    | ["Suchir Krishna Addagondla"] | ["M"]        | ["1343190"]    | ["Fausto Ku"] | ["M"]        | [{"Set_Team1": "21", "Set_Team2": "12"}, {"Set_Team1": "21", "Set_Team2": "14"}] |

### Formats

- Date: Always dd/mm/YY - Zero padded. 20/01/2024 

- Age: U{age} OR Senior - U9 -> U19, Senior

- Level: Tier 4, Bronze, Silver, Gold, Nationals, Restricted.

- Event: OS, WS, OD, WD, XD. all 'ms' or 'md' converts to Open

- Round: Round X, Round {16, 32, ect..}, Quarter-Final, Semi-Final, Final

- TeamX_Info: List of Strings
    - Names: [Name 1, Name 2]
    - Gender: ['M', 'F']
    - MemberID: [ID 1, ID 2]

- Set Scores: List of dict's for each set. [{"Set_Team1": "21", "Set_Team2": "12"}, {"Set_Team1": "21", "Set_Team2": "14"}]